<a href="https://colab.research.google.com/github/ManasaShankaraMurthy/Market-Basket-Analysis/blob/main/MARKET_BASKET_ANALYSIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [331]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [332]:
!pip uninstall mlxtend

Found existing installation: mlxtend 0.21.0
Uninstalling mlxtend-0.21.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/mlxtend-0.21.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/mlxtend/*
Proceed (y/n)? y
  Successfully uninstalled mlxtend-0.21.0


In [333]:
!pip install mlxtend

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached mlxtend-0.21.0-py2.py3-none-any.whl (1.3 MB)


## Importing necessary libraries

In [334]:
import warnings
warnings.filterwarnings('ignore')

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pandas as pd
import time

from mlxtend.preprocessing import TransactionEncoder


## Downloading Kaggle Dataset

In [335]:
#kaggle api token
import os

os.environ["KAGGLE_USERNAME"] = "manasasmurthy"

os.environ["KAGGLE_KEY"] = "981daa73d3babd6acc976c11d5751d3b"

In [336]:
!kaggle datasets download bwandowando/ukraine-russian-crisis-twitter-dataset-1-2-m-rows --unzip

100% 12.6G/12.6G [01:24<00:00, 215MB/s]
100% 12.6G/12.6G [01:24<00:00, 161MB/s]


In [337]:
 # decompressing the file and saving it for future use

import gzip
import shutil

with gzip.open('/content/UkraineWar/UkraineWar/0401_UkraineCombinedTweetsDeduped.csv.gzip', 'rb') as f_in:
    with open('/content/tweets.csv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [338]:
#downloading stopwords

#file contains a dictionary
# key   : language code
# value : list of stopwords

!kaggle datasets download manasasmurthy/stopwords --unzip

  0% 0.00/85.6k [00:00<?, ?B/s]
100% 85.6k/85.6k [00:00<00:00, 47.8MB/s]


In [339]:
import pickle
with open('/content/STOPWORDS.pk','rb') as f:
    stopwords=pickle.load(f)

In [340]:
 #reading the saved file
data=pd.read_csv('/content/tweets.csv')
data=data[['text','language']]

In [341]:
#Creating dictionary to record time taken by the model

result={'language':[],'algorithm':[],'supp':[],'conf':[],'time':[]}

In [342]:
#spark session
spark = SparkSession.builder.master("local[1]").appName("MarketBasketAnalysis").getOrCreate()

# Main


CODES:
['en', 'ar', 'es', 'tr', 'fr', 'no', 'uk', 'hi', 'de', 'und', 'it','zh', 'in', 'pt', 'ja', 'nl', 'th', 'cs', 'bn', 'ur', 'pl', 'sv','eu', 'ru', 'sl', 'et', 'el', 'ro', 'ht', 'fa', 'da', 'or', 'lt','iw', 'gu', 'tl', 'bg', 'fi', 'lv', 'ko', 'ca', 'cy', 'ta', 'te','hu', 'mr', 'sr', 'am', 'ne', 'ml', 'vi', 'is', 'kn', 'ka', 'pa','my', 'ckb', 'hy', 'si', 'ps', 'dv']

In [440]:

language=input("enter language code: ")


enter language code: ru


In [441]:
df=data[data.language==language]
df=df.drop('language',axis=1)

#for faster computation
if df.shape[0]>3000:
  df=df.sample(n=3000)

In [442]:
spark_df=spark.createDataFrame(df)

# Text Preprocessing 

In [443]:
"""
converting entire text to lowercase
removing punctuation marks
removing emojis
removing hyperlinks
removing extra spaces 
removing stopwords

"""

spark_df=spark_df.select("*", F.lower("text").alias('text1'))
spark_df=spark_df.withColumn("text2", F.regexp_replace("text1", "[!,@,#,$,%,^,&,*,;,:,',.,<,>,?,’,‘,“,`,”,–,~,‼,⇅,\",\+,\-,=,\d,\|]", ""))
spark_df=spark_df.withColumn("text3", F.regexp_replace("text2", "\n", " "))
spark_df=spark_df.withColumn("text4", F.regexp_replace("text3", "_", " "))

spark_df=spark_df.withColumn('text5',F.regexp_replace("text4", "[((\U0001F600-\U0001F92F|\U0001F300-\U0001F5FF|\U0001F680-\U0001F6FF|\U0001F190-\U0001F1FF|\U00002702-\U000027B0|\U0001F926-\U0001FA9F|\u200d|\u2640-\u2642|\u2600-\u2B55|\u23cf|\u23e9|\u231a|\ufe0f))]+", ""))


spark_df=spark_df.withColumn("text6",F.trim("text5"))
spark_df=spark_df.withColumn("text7", F.regexp_replace("text6", "\s+", " "))



#stopwords are not available for few languages

if language in list(stopwords.keys()):
  remove_stop = F.udf(lambda x: ' '.join([word for word in list(set(x.split())) if not (word in stopwords[language] or word.startswith('http') or len(word)<3) ]))
else:
  remove_stop = F.udf(lambda x: ' '.join([word for word in list(set(x.split())) if not (word.startswith('http')  or len(word)<3 )]))  

spark_df=spark_df.withColumn('text8', remove_stop(spark_df.text7)) 
spark_df = spark_df.withColumn("text8", F.split("text8", " "))

spark_df=spark_df.drop('text','text1','text2','text3','text4','text5','text6','text7')
spark_df=spark_df.withColumnRenamed('text8', 'text')

spark_df.show()

+--------------------+
|                text|
+--------------------+
|[trip, соединяет,...|
|[мире, нации, три...|
|[вячеслав, спецна...|
|[память, вов, пам...|
|[russianukrainian...|
|[войну, послушает...|
|[украины, вернулс...|
|[уничтоженные, ок...|
|[подростка, комар...|
|[обл, черниговска...|
|[протягивают, кры...|
|[russianarmy, окк...|
|[колонны, российс...|
|[кадыровские, деш...|
|[ukrainewar, ukra...|
|[ебать, russianuk...|
|[брошенную, украи...|
|[границе, глава, ...|
|[украинской, drsj...|
|[слава, stopputin...|
+--------------------+
only showing top 20 rows



In [444]:
#Transaction encoder for mlxtend models

text_list=list(spark_df.select('text').toPandas()['text'])

te=TransactionEncoder()
te_array=te.fit(text_list).transform(text_list)
txn=pd.DataFrame(te_array,columns=te.columns_)


# Frequency of the words

In [445]:

words = (spark_df.select(F.explode(spark_df.text).alias('word')))
word_count=words.groupBy('word').count()
word_count=word_count.orderBy("count", ascending=0)
word_count.show()


+-------------------+-----+
|               word|count|
+-------------------+-----+
|            ukraine| 1203|
|russianukrainianwar|  778|
|         ukrainewar|  722|
|            украина|  687|
|         stoprussia|  627|
|  ukrainerussianwar|  611|
|     войнасукраиной|  377|
|    вторжениероссии|  376|
|     агрессияроссии|  359|
|                всу|  354|
|        войнапутина|  342|
|            украине|  299|
|           белгород|  297|
|             russia|  283|
|   standwithukraine|  235|
|   russiaukrainewar|  235|
|         оккупантов|  226|
|            новости|  192|
|         российские|  191|
|               news|  190|
+-------------------+-----+
only showing top 20 rows



# **Using mlxtend library : **

# 1. Apriori Implementation






In [458]:

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

supp=float(input("Enter min support    : "))
conf=float(input("Enter min confidence : "))

start = time.time()
freq_ap=apriori(txn, min_support=supp,max_len=3, use_colnames=True)
end = time.time()
rules_apriori=association_rules(freq_ap,metric="confidence",min_threshold=conf)
result['language'].append(language)
result['algorithm'].append('apriori')
result['supp'].append(supp)
result['conf'].append(conf)
result['time'].append(end-start)


Enter min support    : 0.1
Enter min confidence : 0.3


In [459]:
freq_ap

,support,itemsets
0,0.259333,(russianukrainianwar)
1,0.209000,(stoprussia)
2,0.401000,(ukraine)
3,0.203667,(ukrainerussianwar)
4,0.240667,(ukrainewar)
5,0.119667,(агрессияроссии)
6,0.114000,(войнапутина)
7,0.125667,(войнасукраиной)
8,0.118000,(всу)
9,0.125333,(вторжениероссии)


In [460]:
rules_apriori

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(stoprussia),(агрессияроссии),0.209000,0.119667,0.118333,0.566188,4.731378,0.093323,2.029298
1,(агрессияроссии),(stoprussia),0.119667,0.209000,0.118333,0.988858,4.731378,0.093323,70.992250
2,(войнапутина),(stoprussia),0.114000,0.209000,0.114000,1.000000,4.784689,0.090174,inf
3,(stoprussia),(войнапутина),0.209000,0.114000,0.114000,0.545455,4.784689,0.090174,1.949200
4,(войнасукраиной),(stoprussia),0.125667,0.209000,0.123000,0.978780,4.683157,0.096736,37.275875
...,...,...,...,...,...,...,...,...,...
157,"(войнасукраиной, украина)",(вторжениероссии),0.114333,0.125333,0.114000,0.997085,7.955462,0.099670,300.010667
158,"(вторжениероссии, украина)",(войнасукраиной),0.115667,0.125667,0.114000,0.985591,7.842897,0.099465,60.678733
159,(войнасукраиной),"(вторжениероссии, украина)",0.125667,0.115667,0.114000,0.907162,7.842897,0.099465,9.525533
160,(вторжениероссии),"(войнасукраиной, украина)",0.125333,0.114333,0.114000,0.909574,7.955462,0.099670,9.794431


# 2. FP-Growth Implementation

In [461]:
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules

supp=float(input("Enter min support    : "))
conf=float(input("Enter min confidence : "))


start = time.time()
freq_fp=fpgrowth(txn, min_support=supp,max_len=3, use_colnames=True)
end = time.time()
rules_fp=association_rules(freq_fp,metric="confidence",min_threshold=conf)
result['language'].append(language)
result['algorithm'].append('fpgrowth')
result['supp'].append(supp)
result['conf'].append(conf)
result['time'].append(end-start)


Enter min support    : 0.15
Enter min confidence : 0.3


In [462]:
freq_fp

,support,itemsets
0,0.229000,(украина)
1,0.209000,(stoprussia)
2,0.401000,(ukraine)
3,0.240667,(ukrainewar)
4,0.203667,(ukrainerussianwar)
5,0.259333,(russianukrainianwar)
6,0.230667,"(ukraine, ukrainewar)"
7,0.200667,"(ukraine, ukrainerussianwar)"
8,0.195333,"(ukrainewar, ukrainerussianwar)"
9,0.194333,"(ukraine, ukrainewar, ukrainerussianwar)"


In [463]:
rules_fp

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ukraine),(ukrainewar),0.401000,0.240667,0.230667,0.575229,2.390147,0.134159,1.787628
1,(ukrainewar),(ukraine),0.240667,0.401000,0.230667,0.958449,2.390147,0.134159,14.415933
2,(ukraine),(ukrainerussianwar),0.401000,0.203667,0.200667,0.500416,2.457033,0.118996,1.593992
3,(ukrainerussianwar),(ukraine),0.203667,0.401000,0.200667,0.985270,2.457033,0.118996,40.665444
4,(ukrainewar),(ukrainerussianwar),0.240667,0.203667,0.195333,0.811634,3.985111,0.146318,4.227593
5,(ukrainerussianwar),(ukrainewar),0.203667,0.240667,0.195333,0.959083,3.985111,0.146318,18.558107
6,"(ukraine, ukrainewar)",(ukrainerussianwar),0.230667,0.203667,0.194333,0.842486,4.136590,0.147354,5.055621
7,"(ukraine, ukrainerussianwar)",(ukrainewar),0.200667,0.240667,0.194333,0.968439,4.023983,0.146040,24.058877
8,"(ukrainewar, ukrainerussianwar)",(ukraine),0.195333,0.401000,0.194333,0.994881,2.480999,0.116005,117.004667
9,(ukraine),"(ukrainewar, ukrainerussianwar)",0.401000,0.195333,0.194333,0.484622,2.480999,0.116005,1.561313


# Experiments

In [464]:
experiment=[freq_ap,freq_fp]
algo=int(input("Enter 0: for selecting apriori frequent itemset\nEnter 1: for selecting fpgrowth frequent itemset\n"))

Enter 0: for selecting apriori frequent itemset
Enter 1: for selecting fpgrowth frequent itemset
0


# a) Singleton

In [465]:
experiment[algo][experiment[algo].itemsets.apply(lambda x: len(x)==1)]

,support,itemsets
0,0.259333,(russianukrainianwar)
1,0.209000,(stoprussia)
2,0.401000,(ukraine)
3,0.203667,(ukrainerussianwar)
4,0.240667,(ukrainewar)
5,0.119667,(агрессияроссии)
6,0.114000,(войнапутина)
7,0.125667,(войнасукраиной)
8,0.118000,(всу)
9,0.125333,(вторжениероссии)


# b) Pair

In [466]:
experiment[algo][experiment[algo].itemsets.apply(lambda x: len(x)==2)]

,support,itemsets
11,0.118333,"(stoprussia, агрессияроссии)"
12,0.114000,"(войнапутина, stoprussia)"
13,0.123000,"(войнасукраиной, stoprussia)"
14,0.123000,"(вторжениероссии, stoprussia)"
15,0.126333,"(stoprussia, украина)"
16,0.200667,"(ukraine, ukrainerussianwar)"
17,0.230667,"(ukraine, ukrainewar)"
18,0.195333,"(ukrainewar, ukrainerussianwar)"
19,0.114000,"(войнапутина, агрессияроссии)"
20,0.114000,"(войнасукраиной, агрессияроссии)"


# c) Triplet

In [467]:
experiment[algo][experiment[algo].itemsets.apply(lambda x: len(x)==3)]

,support,itemsets
29,0.114000,"(войнапутина, stoprussia, агрессияроссии)"
30,0.113667,"(войнасукраиной, stoprussia, агрессияроссии)"
31,0.113667,"(вторжениероссии, stoprussia, агрессияроссии)"
32,0.115000,"(украина, stoprussia, агрессияроссии)"
33,0.113667,"(войнасукраиной, войнапутина, stoprussia)"
34,0.113667,"(вторжениероссии, войнапутина, stoprussia)"
35,0.114000,"(войнапутина, stoprussia, украина)"
36,0.123000,"(войнасукраиной, вторжениероссии, stoprussia)"
37,0.114000,"(войнасукраиной, stoprussia, украина)"
38,0.114000,"(вторжениероссии, stoprussia, украина)"


# Final Results

In [468]:
result_df=pd.DataFrame(result)
result_df

,language,algorithm,supp,conf,time
0,en,apriori,0.10,0.30,0.050534
1,en,fpgrowth,0.10,0.30,0.073038
2,en,apriori,0.20,0.35,0.049082
3,en,fpgrowth,0.23,0.40,0.068010
4,en,apriori,0.35,0.60,0.048327
5,en,fpgrowth,0.38,0.43,0.069275
6,it,apriori,0.30,0.50,0.046006
7,it,fpgrowth,0.45,0.60,0.088620
8,it,apriori,0.10,0.30,0.052300
9,it,fpgrowth,0.10,0.35,0.073429


In [469]:
import plotly.express as px


fig = px.scatter(result_df, x="supp",y="time",animation_frame='language',color='algorithm' ,title="time v/s support")


fig.show()